In [1]:
%config InlineBackend.figure_format = 'retina'

In [30]:
import numpy as np
import os
import glob
from pathlib import Path
from matplotlib import pyplot as plt
from astropy.io import ascii, fits
from astropy.table import Table,Column
from datetime import datetime
from scipy.optimize import curve_fit
from scipy.ndimage import median_filter
from scipy.interpolate import UnivariateSpline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.feature import peak_local_max
from astropy.stats import sigma_clip, gaussian_fwhm_to_sigma
from numpy.polynomial.chebyshev import chebfit, chebval
from astropy.modeling.models import Gaussian1D, Chebyshev2D
from astropy.modeling.fitting import LevMarLSQFitter
from matplotlib import gridspec, rcParams, rc
from IPython.display import Image
import preproc 

# plt.rcParams['axes.linewidth'] = 2
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14

WD = Path('./data/proj5')
RAWDIR = WD / 'raw' / 'P03hbahk0304145443GZ_data'
OUTDIR = WD / 'out'

if not OUTDIR.exists():
    OUTDIR.mkdir(parents=True)


In [31]:
def make_summary_table_focas(rawdir, suffix='.fits.gz'):
    """Make a summary table of the raw data in the directory. This function is
    specifically designed for the data taken with the FOCAS instrument.

    Args:
        rawdir (pathlib.Path): The directory containing the raw data.
        suffix (str, optional): The suffix of the raw data files. Defaults to
            '.fits.gz'.

    Returns:
        stab (astropy.table.Table): The summary table of the raw data in the directory.
        
    """
    # making a summary table
    summary = []
    for f in rawdir.glob('*'+suffix):
        hdr = fits.getheader(f)
        
        # getting the filter information from the header
        filt1 = hdr['FILTER01']
        filt2 = hdr['FILTER02']
        filt3 = hdr['FILTER03']
            
        X = hdr['AIRMASS'] # airmass
        disp = hdr['DISPERSR'] # disperser
        
        # appending the data to the summary list
        summary.append([f.name, hdr['DATE-OBS'], hdr['OBJECT'], hdr['RA'], hdr['DEC'],
                        hdr['DATA-TYP'], hdr['EXPTIME'], X, disp, hdr['SLT-WID'],
                        hdr['SLT-PA'], hdr['SLIT'], filt1, filt2, filt3, hdr['UT-STR'],
                        hdr['UT-END']])
        
    # creating the summary table
    stab = Table(rows=summary,
                 names=['filename', 'date-obs', 'object', 'ra', 'dec', 'type',
                        'exptime', 'airmass', 'disperser', 'slit-width', 'slit-pa',
                        'slit', 'filter1', 'filter2','filter3', 'ut-str', 'ut-end'],
                 dtype=['U50', 'U50', 'U50', 'U50', 'U50', 'U50', 'f8', 'f8', 'U50',
                        'f8', 'f8', 'U50', 'U50', 'U50', 'U50', 'U50', 'U50'])
    return stab

summary_table = make_summary_table_focas(RAWDIR)

In [32]:
summary_table.show_in_notebook(display_length=5)

idx,filename,date-obs,object,ra,dec,type,exptime,airmass,disperser,slit-width,slit-pa,slit,filter1,filter2,filter3,ut-str,ut-end
0,FCSA00141794.fits.gz,2013-06-07,DOMEFLAT,11:17:48.038,+19:54:08.26,DOMEFLAT,3.0,1.0,SCFCGRHD90,0.5,-90.3,SCFCSLLC08,NONE,SCFCFLSO58,NONE,04:37:11.906,04:37:15.139
1,FCSA00141920.fits.gz,2013-06-07,GRB 130606A,16:37:41.157,+29:47:45.71,OBJECT,1200.0,1.136,SCFCGRHD90,0.5,-90.3,SCFCSLLC08,NONE,SCFCFLSO58,NONE,07:59:00.676,08:19:01.117
2,FCSA00141930.fits.gz,2013-06-07,GRB 130606A,16:37:41.230,+29:47:45.13,OBJECT,1200.0,1.038,SCFCGRHD90,0.5,-90.3,SCFCSLLC08,NONE,SCFCFLSO58,NONE,09:03:28.980,09:23:29.405
3,FCSA00141848.fits.gz,2013-06-07,BIAS,11:39:50.841,+19:54:11.18,BIAS,0.0,1.0,SCFCGRMR01,0.5,-90.3,SCFCSLLC20,NONE,SCFCFLSO58,NONE,04:59:10.571,04:59:10.574
4,FCSA00141916.fits.gz,2013-06-07,GRB 130606A,16:37:41.020,+29:47:44.75,OBJECT,1200.0,1.254,SCFCGRHD90,0.5,-90.3,SCFCSLLC08,NONE,SCFCFLSO58,NONE,07:18:00.165,07:38:00.608
5,FCSA00141734.fits.gz,2013-06-06,BIAS,21:41:03.767,+17:46:28.41,BIAS,0.0,1.001,SCFCGRHD90,0.5,-0.3,SCFCSLLC20,NONE,SCFCFLSO58,NONE,14:53:03.807,14:53:03.810
6,FCSA00141566.fits.gz,2013-06-06,DOMEFLAT,11:23:03.374,+19:54:09.25,DOMEFLAT,1.1,1.0,SCFCGRHD90,0.5,-90.3,SCFCSLLC20,NONE,SCFCFLSO58,NONE,04:46:28.971,04:46:30.721
7,FCSA00141576.fits.gz,2013-06-06,DOMEFLAT,11:26:18.779,+19:54:09.76,DOMEFLAT,3.0,1.0,SCFCGRHD90,0.5,-90.3,SCFCSLLC08,NONE,SCFCFLSO58,NONE,04:49:43.639,04:49:46.993
8,FCSA00141822.fits.gz,2013-06-07,DOMEFLAT,11:26:05.681,+19:54:09.65,DOMEFLAT,1.1,1.0,SCFCGRHD90,0.5,-90.3,SCFCSLLC20,NONE,SCFCFLSO58,NONE,04:45:27.990,04:45:29.664
9,FCSA00141582.fits.gz,2013-06-06,COMPARISON,11:30:54.937,+19:54:10.37,COMPARISON,1.0,1.0,SCFCGRHD90,0.5,-90.3,SCFCSLLC20,NONE,SCFCFLSO58,NONE,04:54:18.901,04:54:20.208


In [33]:
biastab = summary_table[summary_table['type'] == 'BIAS']
flattab = summary_table[summary_table['type'] == 'DOMEFLAT']
arctab = summary_table[summary_table['type'] == 'COMPARISON']
scitab = summary_table[summary_table['type'] == 'OBJECT']

biaslist = [RAWDIR / f for f in biastab['filename']]
mbias = preproc.combine_bias(biaslist, outdir=OUTDIR, outname='MBias.fits')

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial 

this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T04:59:10.645' from MJD-END'. [astropy.wcs.wcs]
  'degree' -> 'deg'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T14:53:03.880' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T04:56:58.652' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T04:59:35.780' from MJD-END'. [astropy.wcs.wcs]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial 

Set DATE-END to '2013-06-07T15:06:35.545' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T05:00:00.751' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T04:56:34.054' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T15:07:00.686' from MJD-END'. [astropy.wcs.wcs]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial 

Set DATE-END to '2013-06-07T15:07:53.456' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T04:56:09.042' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T14:53:29.035' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T15:06:10.558' from MJD-END'. [astropy.wcs.wcs]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial 

Set DATE-END to '2013-06-06T04:55:19.314' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T04:58:45.726' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T14:52:38.925' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T15:07:26.756' from MJD-END'. [astropy.wcs.wcs]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial axes.
 [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: An exception happened while extracting WCS information from the Header.
InconsistentAxisTypesError: ERROR 4 in wcs_types() at line 3105 of file cextern/wcslib/C/wcs.c:
Unmatched celestial 

Set DATE-END to '2013-06-06T14:52:13.481' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-07T05:00:25.744' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T04:55:43.954' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2013-06-06T14:51:48.267' from MJD-END'. [astropy.wcs.wcs]
